# Model Quality: Code Practice

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn import ensemble

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import ClassificationPreset, RegressionPreset
from evidently.metrics import *

## Classification Quality

In [ ]:
#Dataset for binary probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame=True) #auto
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

In [ ]:
#Dataset for regression
housing_data = datasets.fetch_california_housing(as_frame=True) #auto 
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [ ]:
classification_performance_report = Report(metrics=[
    ClassificationPreset(probas_threshold=0.7),
])

classification_performance_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)

classification_performance_report.show(mode='inline')

In [ ]:
classification_report = Report(metrics=[
    ClassificationQualityMetric(),
    ClassificationClassBalance(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    ClassificationConfusionMatrix(),
    ClassificationQualityByClass(),
    ClassificationClassSeparationPlot(),
    ClassificationProbDistribution(),
    ClassificationRocCurve(),
    ClassificationPRCurve(),
    ClassificationPRTable(),
    ClassificationQualityByFeatureTable(columns=['mean area', 'fractal dimension error']),

    
])

classification_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
classification_report.show(mode='inline')

In [ ]:
classification_report.save_html('classification_report.html')

In [ ]:
classification_report.json()

In [ ]:
classification_report.as_dict()

## Regression Quality

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=housing_ref.sort_index(), current_data=housing_cur.sort_index())
regression_performance_report.show(mode='inline')

In [ ]:
regression_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionPredictedVsActualScatter(),
    RegressionPredictedVsActualPlot(),
    RegressionErrorPlot(),
    RegressionAbsPercentageErrorPlot(),
    RegressionErrorDistribution(),
    RegressionErrorNormality(),
    RegressionTopErrorMetric(),
    RegressionErrorBiasTable(columns=['MedInc', 'AveRooms']),
    
])

regression_report.run(reference_data=housing_ref, current_data=housing_cur)
regression_report.show(mode='inline')